In [ ]:
#@markdown <h1>Rodar websocket</h1>
# WebSocket no Google Colab com ngrok

# Instalar dependências
!pip install pyngrok websockets nest_asyncio

import asyncio
import websockets
import nest_asyncio
from pyngrok import ngrok
import uuid
import json  # Para manipular JSON

# Permitir execução de loops assíncronos no Colab
nest_asyncio.apply()

# Configurar token do ngrok (substitua 'SEU_TOKEN_AQUI' pelo seu token real)
ngrok.set_auth_token("2uTAJZmY4WE0TdhMSjSP6s2zMQB_3J7fXZCSzNJYMxPcbqqKk")

# Lista de conexões ativas
clients = {}

# Criar servidor WebSocket
async def echo(websocket):
    # Atribui um ID único para cada usuário
    user_id = str(uuid.uuid4())  # Cria um ID único para cada cliente
    print(f"Novo cliente conectado: {user_id}")

    # Armazena a conexão do usuário
    clients[user_id] = websocket

    try:
        async for values in websocket:
            print(values)
            if not values.strip():  # Se for vazio ou apenas espaços, ignora
                continue

            try:
                values = json.loads(values)  # Converte a string JSON para dicionário
            except json.JSONDecodeError:
                print(f"Erro ao decodificar JSON: {values}")
                continue  # Pula para a próxima iteração

            user = values.get("user", user_id)  # Se o user não for enviado, usa o ID gerado
            message = values.get("message", "")

            print(f"Recebido de {user}: {message}")

            # Envia a mensagem para todos os clientes conectados
            for client_id, client in clients.items():
                if client != websocket:  # Não envia de volta para quem mandou
                    await client.send(json.dumps({"user": user, "message": message}))
    except websockets.exceptions.ConnectionClosed:
        print(f"Cliente {user_id} desconectado")
    finally:
        # Remove o cliente da lista de conexões quando ele se desconectar
        del clients[user_id]

# Iniciar WebSocket na porta 8765
PORT = 8765
start_server = websockets.serve(echo, "0.0.0.0", PORT)

# Criar um túnel com ngrok
public_url = ngrok.connect(PORT, "http")
print(f"URL do WebSocket: {public_url}")

# Rodar servidor WebSocket
loop = asyncio.get_event_loop()
loop.run_until_complete(start_server)
loop.run_forever()


URL do WebSocket: NgrokTunnel: "https://a398-34-105-27-142.ngrok-free.app" -> "http://localhost:8765"
Novo cliente conectado: c291f095-1a57-4156-a337-7f0873210c84
Novo cliente conectado: d42d1912-8df5-4fbc-b09a-fa14730dc1c1
{"user":"Jony","message":"Olá Midoria!"}
Recebido de Jony: Olá Midoria!
{"user":"goku","message":"Opa Jony!"}
Recebido de goku: Opa Jony!
{"user":"Jony","message":"Ata pow! É você né goku?"}
Recebido de Jony: Ata pow! É você né goku?
Novo cliente conectado: a3e4c5b7-016b-4f3d-9114-4c475c4a5625
Novo cliente conectado: 9b06aec5-1b24-4437-aefb-75c975c8b16a
Novo cliente conectado: d11086a4-df12-42a8-91f4-6e19b699ebf0
Novo cliente conectado: ff58a853-07f5-45f0-a065-29afc5192d97
Novo cliente conectado: c35b991e-4622-4b57-bff5-0e3ae9aa400a
Novo cliente conectado: 95f7ba75-25a2-4e7e-9938-9e8fb059024c
Novo cliente conectado: f61c04e4-11a4-4cc2-988c-c654cec8dab9
Novo cliente conectado: 8daea302-94a0-42f0-9b80-a877dba628fd
Novo cliente conectado: 776c5c99-a235-421b-8451-d61c8a

In [ ]:
#@markdown <h2>Desconectando da porta 8765 para reiniciar o websocket<h2>
import os
os.system("fuser -k 8765/tcp")